<h1>Question 3</h1>
<h2>Explore and cluster the neighborhoods in Toronto</h2>

In [51]:
# data analsysis
import pandas as pd 
# data in a vectorized manner manipulation
import numpy as np 
# tranform JSON file
from pandas.io.json import json_normalize 
# JSON files manipulation
import json 
# HTTP library
import requests 
# scraping library
from bs4 import BeautifulSoup 
# sklearn clustering algorithm
from sklearn.cluster import KMeans 
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
# folium
import folium
# convert an address into latitude and longitude values 
from geopy.geocoders import Nominatim 

print('Libraries imported.')

Libraries imported.


In [122]:
postcodes_with_coordinates_df = pd.read_csv('data/toronto-postalcode-latlong.csv')
postcodes_with_coordinates_df.drop(["Unnamed: 0"], axis=1, inplace=True)

print('The postalcode with coordinates dataframe shape is', postcodes_with_coordinates_df.shape)
postcodes_with_coordinates_df.head(12)

The postalcode with coordinates dataframe shape is (103, 5)


,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


In [134]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address, timeout=15)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [135]:
#latitude_toronto = 43.653963
#longitude_toronto = -79.387207
df_toronto = postcodes_with_coordinates_df
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(
    df_toronto['Latitude'], 
    df_toronto['Longitude'], 
    df_toronto['Borough'], 
    df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

The map shows that the Postal codes are not evenly spaced, and the area cover by some of them, using a radius of 500 meters, overlaps. A different radius for each postcode results in a better venues search because that will avoid misrepresentation of the number of venues per postcode caused by too large or low radius values.

Next thing to do is explore each Postcode to get venues using the Foursquare API.

In [130]:
# Foursquare ID
CLIENT_ID = 'RT3CZHJPAA3QVQLSBQMWDKUFWSE44IK5KPUS3CJLUVJ4ZZ0B'
# Foursquare Secret
CLIENT_SECRET = 'KRAUWAKS1FJCWQC0K5U4JPWTMXAUTF4QOGRHSPL3NLUCUPE0' 
VERSION = '20181102'

In [149]:
# defining radius and limit of venues
radius = 500
LIMIT = 100

# function
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}' \
            '&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
        'Neighbourhood', 
        'Neighbourhood Latitude', 
        'Neighbourhood Longitude', 
        'Venue', 
        'Venue Latitude', 
        'Venue Longitude', 
        'Venue Category']
    return(nearby_venues)

In [151]:
toronto_venues = getNearbyVenues(
    names = df_toronto['Neighbourhood'],
    latitudes = df_toronto['Latitude'],
    longitudes = df_toronto['Longitude']
)

Parkwoods
Victoria Village
Harbourfront
Lawrence Heights, Lawrence Manor
Queen's Park
Queen's Park
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin


ConnectionError: HTTPSConnectionPool(host='api.foursquare.com', port=443): Max retries exceeded with url: /v2/venues/explore?&client_id=RT3CZHJPAA3QVQLSBQMWDKUFWSE44IK5KPUS3CJLUVJ4ZZ0B&client_secret=KRAUWAKS1FJCWQC0K5U4JPWTMXAUTF4QOGRHSPL3NLUCUPE0&v=20181102&ll=43.66900510000001,-79.4422593&radius=500&limit=100 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000025CE43DD828>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [154]:
print('toronto venues dataframe shape is', toronto_venues.shape)
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
toronto_venues.head()

toronto venues dataframe shape is (2238, 7)
There are 274 uniques categories.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,Bella Vita Catering & Private Chef Service,43.756651,-79.331524,BBQ Joint
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


Analyze Each Neighborhood

In [156]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighborhood'] 
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print('toronto onehot dataframe shape is', toronto_onehot.shape)
toronto_onehot.head()

toronto onehot dataframe shape is (2238, 275)


,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [159]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
print('toronto grouped dataframe shape is', toronto_grouped.shape)
toronto_grouped.head()

toronto grouped dataframe shape is (99, 275)


,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0,...,0.02,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0


each neighborhood along with the top 5 most common venues

In [162]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
              venue  freq
0       Coffee Shop  0.08
1              Café  0.05
2               Bar  0.04
3        Steakhouse  0.04
4  Asian Restaurant  0.03


----Agincourt----
                       venue  freq
0             Breakfast Spot   0.2
1                     Lounge   0.2
2             Clothing Store   0.2
3  Latin American Restaurant   0.2
4               Skating Rink   0.2


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                       venue  freq
0                 Playground  0.33
1           Sculpture Garden  0.33
2                       Park  0.33
3  Middle Eastern Restaurant  0.00
4        Monument / Landmark  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0           Pizza Place  0.22
1         Grocery Store  0.22
2   Fried Chicken Joint  0.11
3              Pharmacy  0.11
4  Fast Food Restaurant  0.11


--

                        venue  freq
0  Construction & Landscaping  0.33
1                        Park  0.33
2                      Bakery  0.33
3           Accessories Store  0.00
4          Miscellaneous Shop  0.00


----East Birchmount Park, Ionview, Kennedy Park----
              venue  freq
0    Discount Store   0.2
1       Bus Station   0.2
2  Department Store   0.2
3       Coffee Shop   0.2
4        Hobby Shop   0.2


----East Toronto----
               venue  freq
0      Metro Station   0.2
1        Pizza Place   0.2
2  Convenience Store   0.2
3        Coffee Shop   0.2
4               Park   0.2


----Emery, Humberlea----
                 venue  freq
0       Baseball Field   1.0
1    Accessories Store   0.0
2   Miscellaneous Shop   0.0
3                Motel   0.0
4  Monument / Landmark   0.0


----Fairview, Henry Farm, Oriole----
                  venue  freq
0        Clothing Store  0.16
1  Fast Food Restaurant  0.08
2           Coffee Shop  0.08
3   Japanese Restaurant  0.05

                       venue  freq
0                Coffee Shop  0.08
1             Clothing Store  0.07
2                       Café  0.04
3             Cosmetics Shop  0.04
4  Middle Eastern Restaurant  0.03


----Scarborough Village----
                venue  freq
0          Playground   0.5
1      Cosmetics Shop   0.5
2  Miscellaneous Shop   0.0
3       Movie Theater   0.0
4               Motel   0.0


----Silver Hills, York Mills----
               venue  freq
0          Cafeteria   1.0
1  Accessories Store   0.0
2  Mobile Phone Shop   0.0
3      Movie Theater   0.0
4              Motel   0.0


----St. James Town----
         venue  freq
0         Café  0.06
1  Coffee Shop  0.06
2   Restaurant  0.05
3        Hotel  0.04
4       Bakery  0.03


----Stn A PO Boxes 25 The Esplanade----
                venue  freq
0         Coffee Shop  0.11
1          Restaurant  0.04
2                Café  0.04
3  Seafood Restaurant  0.03
4  Italian Restaurant  0.03


----Studio District----
        

new dataframe and display the top 10 venues for each neighborhood.

In [163]:
# function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [166]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

print('neighbourhoods venues dataframe shape is', toronto_grouped.shape)
neighborhoods_venues_sorted.head()

neighbourhoods venues dataframe shape is (99, 275)


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Restaurant,Thai Restaurant,Sushi Restaurant,Burger Joint,Asian Restaurant,Bakery
1,Agincourt,Lounge,Breakfast Spot,Latin American Restaurant,Skating Rink,Clothing Store,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Sculpture Garden,Park,Yoga Studio,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Pizza Place,Grocery Store,Pharmacy,Fried Chicken Joint,Beer Store,Sandwich Place,Fast Food Restaurant,Construction & Landscaping,Convenience Store,Colombian Restaurant
4,"Alderwood, Long Branch",Pizza Place,Gym,Pharmacy,Sandwich Place,Skating Rink,Coffee Shop,Athletics & Sports,Pub,Donut Shop,Diner


<h2>Run k-means to cluster the neighborhood into 5 clusters.</h2>

In [167]:
# set number of clusters
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 1, 3, 3, 0, 0, 0, 0, 0])

In [168]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = postcodes_with_coordinates_df
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,BBQ Joint,Park,Food & Drink Shop,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,French Restaurant,Portuguese Restaurant,Hockey Arena,Coffee Shop,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0.0,Coffee Shop,Bakery,Park,Pub,Mexican Restaurant,Restaurant,Breakfast Spot,Café,Chocolate Shop,Dessert Shop
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,0.0,Clothing Store,Accessories Store,Coffee Shop,Shoe Store,Miscellaneous Shop,Carpet Store,Gift Shop,Furniture / Home Store,Boutique,Vietnamese Restaurant
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,0.0,Coffee Shop,Park,Gym,Restaurant,Sushi Restaurant,Bar,Portuguese Restaurant,Mexican Restaurant,Diner,Café


visualize the resulting clusters

In [169]:
# create map
map_clusters = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
    toronto_merged['Latitude'], 
    toronto_merged['Longitude'], 
    toronto_merged['Neighbourhood'], 
    toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #color=rainbow[cluster-1],
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h2>Examine Clusters</h2>

<h3>cluster 1</h3>

In [170]:
toronto_merged.loc[
    toronto_merged['Cluster Labels'] == 0, 
    toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]
]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0.0,French Restaurant,Portuguese Restaurant,Hockey Arena,Coffee Shop,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,Downtown Toronto,0.0,Coffee Shop,Bakery,Park,Pub,Mexican Restaurant,Restaurant,Breakfast Spot,Café,Chocolate Shop,Dessert Shop
3,North York,0.0,Clothing Store,Accessories Store,Coffee Shop,Shoe Store,Miscellaneous Shop,Carpet Store,Gift Shop,Furniture / Home Store,Boutique,Vietnamese Restaurant
4,Queen's Park,0.0,Coffee Shop,Park,Gym,Restaurant,Sushi Restaurant,Bar,Portuguese Restaurant,Mexican Restaurant,Diner,Café
5,Downtown Toronto,0.0,Coffee Shop,Park,Gym,Restaurant,Sushi Restaurant,Bar,Portuguese Restaurant,Mexican Restaurant,Diner,Café
7,North York,0.0,Café,Gym / Fitness Center,Caribbean Restaurant,Japanese Restaurant,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Yoga Studio
9,Downtown Toronto,0.0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Restaurant,Bakery,Bubble Tea Shop,Pizza Place,Sporting Goods Shop
13,North York,0.0,Gym,Beer Store,Coffee Shop,Asian Restaurant,Dim Sum Restaurant,Concert Hall,Bike Shop,Sporting Goods Shop,Japanese Restaurant,Italian Restaurant
15,Downtown Toronto,0.0,Café,Coffee Shop,Restaurant,Hotel,Diner,Beer Bar,Breakfast Spot,Italian Restaurant,Cocktail Bar,Clothing Store
17,Etobicoke,0.0,Liquor Store,Beer Store,Pizza Place,Coffee Shop,Café,Shopping Plaza,Pharmacy,Gluten-free Restaurant,Event Space,Empanada Restaurant


<h3>cluster 2</h3>  

In [171]:
toronto_merged.loc[
    toronto_merged['Cluster Labels'] == 1, 
    toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]
]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1.0,BBQ Joint,Park,Food & Drink Shop,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
16,York,1.0,Field,Trail,Park,Hockey Arena,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
21,York,1.0,Park,Women's Store,Market,Fast Food Restaurant,Construction & Landscaping,Concert Hall,Comic Shop,Farmers Market,Falafel Restaurant,Event Space
32,Scarborough,1.0,Playground,Cosmetics Shop,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
40,North York,1.0,Airport,Park,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
49,North York,1.0,Bakery,Construction & Landscaping,Park,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
61,Central Toronto,1.0,Park,Swim School,Lake,Bus Line,Yoga Studio,Eastern European Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
64,York,1.0,Park,Convenience Store,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
66,North York,1.0,Park,Bank,Bar,Convenience Store,Yoga Studio,Electronics Store,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
68,Central Toronto,1.0,Trail,Park,Sushi Restaurant,Bus Line,Jewelry Store,Yoga Studio,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant


<h3>cluster 3</h3>

In [172]:
toronto_merged.loc[
    toronto_merged['Cluster Labels'] == 2, 
    toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]
]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,North York,2.0,Food Truck,Baseball Field,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Yoga Studio,Discount Store
57,North York,2.0,Baseball Field,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant,Discount Store


<h3>cluster 4</h3>

In [173]:
toronto_merged.loc[
    toronto_merged['Cluster Labels'] == 3, 
    toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]
]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,3.0,Fast Food Restaurant,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
8,East York,3.0,Pizza Place,Fast Food Restaurant,Gastropub,Pet Store,Pharmacy,Intersection,Bank,Athletics & Sports,Gym / Fitness Center,German Restaurant
10,North York,3.0,Pizza Place,Japanese Restaurant,Sushi Restaurant,Asian Restaurant,Pub,Park,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
12,Scarborough,3.0,History Museum,Bar,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
14,East York,3.0,Pharmacy,Video Store,Bus Stop,Asian Restaurant,Curling Ice,Cosmetics Shop,Beer Store,Skating Rink,Park,Ethiopian Restaurant
18,Scarborough,3.0,Pizza Place,Breakfast Spot,Rental Car Location,Medical Center,Electronics Store,Intersection,Mexican Restaurant,Drugstore,Dog Run,Doner Restaurant
19,East Toronto,3.0,Pub,Trail,Health Food Store,Neighborhood,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
26,Scarborough,3.0,Hakka Restaurant,Athletics & Sports,Fried Chicken Joint,Bank,Bakery,Gas Station,Thai Restaurant,Caribbean Restaurant,Cuban Restaurant,Cupcake Shop
27,North York,3.0,Dog Run,Pool,Athletics & Sports,Golf Course,Mediterranean Restaurant,Yoga Studio,Eastern European Restaurant,Doner Restaurant,Donut Shop,Drugstore
29,East York,3.0,Indian Restaurant,Burger Joint,Yoga Studio,Bank,Gym / Fitness Center,Gas Station,Housing Development,Discount Store,Park,Grocery Store


<h3>cluster 5</h3>

In [174]:
toronto_merged.loc[
    toronto_merged['Cluster Labels'] == 4, 
    toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]
]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,North York,4.0,Cafeteria,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Field
